<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/02-WebAPIs/B2-IBM_Watson_Natural_Language_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interacting with the IBM Watson Natural Language Understanding API

Another useful API, especially when dealing with text, is the [IBM Watson  Natural Language Understanding API](https://console.bluemix.net/catalog/services/natural-language-understanding), which offers a variety of text analysis functionalities, such as sentiment analysis, entity extraction, keyword extraction, etc.

We will give a couple of examples below, to understand how we can take an unstructured piece of text (either the text alone, or a URL with text), and perform some analysis.





## Sentiment ana emotion analysis

We will first start with the `/analyze` API call ([documentation](https://cloud.ibm.com/apidocs/natural-language-understanding#analyzeget)), which takes as input a piece of text, and returns an analysis across various dimensions. 

The API supports the following analyses:

`categories,classifications,concepts,emotion,entities,keywords,metadata,relations,semantic_roles,sentiment,summarization (experimental),syntax`

The API supports not only English, but also a [variety of non-English languages](https://cloud.ibm.com/docs/natural-language-understanding?topic=natural-language-understanding-detectable-languages).

In our introductory attempt, we will use the `sentiment` and `emotion` and focus on English texts. 



In [ ]:
import requests

In [ ]:
def analyzeText(text=None, url=None):
  '''
  This function gets as input either a piece of text or a URL
  and then contacts the IBM Watson NLU API to perform sentiment
  and emotion analysis on the text/url.
  '''

  # Pointers to the IBM Server that we will use for the
  # endpointn and the API key provided by IBM. 
  # This is my own, personal IBM_SERVER_URL and IBM_API_KEY
  # 
  # You probably should can register and get your own credentials
  # The ones below have a quota of 1000 calls per day 
  # and can run out quickly if multiple people use these
  IBM_SERVER_URL = 'https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/7955e520-ee3e-4435-917f-f2e626d7b297'
  IBM_API_KEY = 'ipYHoxag1pB7bkURAGAnZUaOstg05h8_jsgL4io_Co1l'

  endpoint = f"{IBM_SERVER_URL}/v1/analyze"
  username = "apikey"
  password = IBM_API_KEY
  
  parameters = {
      'features': 'emotion,sentiment',
      'version' : '2022-04-07',
      'text': text,
      'language' : 'en',
      'url' : url # this is an alternative to sending the text
  }

  resp = requests.get(endpoint, params=parameters, auth=(username, password))
  
  return resp.json()

In [ ]:
# We will analyze the text below using the IBM Watson API

review = '''
I got their Egg & Cheese sandwich on a Whole Wheat Everything Bagel. 
First off, I loved loved loved the texture of the bagel itself. 
It was very chewy yet soft, which is a top feature for a NY style bagel. 
However, I thought there could've been more seasoning on top of 
the bagel as I found the bagel itself to be a bit bland. 

Speaking of bland, I thought the egg and cheese filling were also quite bland. 
This was definitely lacking salt and pepper in the eggs and the cheese didn't
really add too much flavor either, which was really disappointing! 
My mom also had the same complaint with her bagel sandwich 
(she had the egg sandwich on a blueberry bagel) so I definitely wasn't 
the only one.

'''

In [ ]:
data = analyzeText(text=review)
data

Now, let's try to understand the structure of the answer. First, we check the high-level keys.

In [ ]:
data.keys()

Now, let's check the content of these keys:

In [ ]:
data['language']

In [ ]:
data['sentiment']

In [ ]:
data['emotion']

In [ ]:
# Let's go deeper into the 'emotion' dictionary
data['emotion']['document']

In [ ]:
# And a bit more
data['emotion']['document']['emotion']

### Exercise 1

Type your own piece of text, and analyze it to extract sentiment and emotions. Discuss your findings

In [ ]:
my_own_text = '''
'''

analyzeText( text = my_own_text )

### Exercise 2

Below is slightly different call, which takes as input a URL to analyze, instead of a piece of text. Use it to analyze a URL of your choice

In [ ]:
news_url = 'https://www.wsj.com/articles/june-jobs-report-unemployment-rate-economy-growth-2022-11657237512'
analyzeText(url = news_url)

## Entities call

The code below changes slightly the way that we way that we call the API. Instead of asking for sentiment and emotion, we instead ask to extract entities from the text, and also the sentiment and emotion for each of these entities.

In terms of natural language processing, we will examine a couple of capabilities of the API. First, you will see that there is the capability of "normalizing" each entity, so that two different ways of saying the same thing get mapped to the same entity. So for example, "President Trump" and "Donald Trump" get mapped to the same Knowledge Graph entity.

In [ ]:
def extractEntities(text=None, url=None):

    IBM_SERVER_URL = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/9e683088-0d12-4399-8118-518f3e60e8c4'
    IBM_API_KEY = 'yx39wyiwPNGm7DoDUPCSJB4SzFkr0qurARfbGYyEdaoC'

    endpoint = f"{IBM_SERVER_URL}/v1/analyze"
    username = "apikey"
    password = IBM_API_KEY
    
    parameters = {
        'features': 'entities',
        'version' : '2022-04-07',
        'entities.limit' : 10,
        'entities.sentiment' : True,
        'entities.emotion' : True,
        'text': text,
        'language' : 'en',
        'url' : url # this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [ ]:
news_url = 'https://www.nytimes.com/2022/05/21/world/europe/kirill-putin-russian-orthodox-church.html'

data = extractEntities(url=news_url)

In [ ]:
data

In [ ]:
# Let's see what we get back as top-level attributes
data.keys()

In [ ]:
# Let' see the entities list
data["entities"]

In [ ]:
# Let' see the 7th entity. Notice the "disambiguated" attribute that
# points to "canonical" versions of the entity, in DBPedia
# Notice that "Patriarch Kirill"" is the actual term used in the text
data["entities"][6]

In [ ]:
# Let's put the results in a dataframe, so that we can browse easier
import pandas as pd

pd.json_normalize(data['entities'])

### Exercise

* First of all, **get your own credentials for the IBM Watson API**. The demo key that we use above has a limited quota.
* Use an API to get news articles. 
    * Option 1: Use the API at https://newsapi.org to fetch the news from various sources. Print the entities that are currently being discussed in the news, together with their relevance value and the associated sentiment.
    * Option 2: Use the NY Times API to fetch the Top Stories News. You can register and get an API key at https://developer.nytimes.com/. The `Top Stories V2 API` provides the details of the news of the day: (The API call documentation is at https://developer.nytimes.com/docs/top-stories-product/1/overview and the API Call is  https://api.nytimes.com/svc/topstories/v2/home.json?api-key=PUTYOURKEYHERE). Repeat the entity extraction process from above.
    * Option 3: Use the Guardian API at https://open-platform.theguardian.com/documentation/ to fetch news from The Guardian.
